In [1]:
import os
import numpy as np
import sys
from timeit import default_timer as timer

sys.path.append("/Users/hakunahahannah/Documents/Projects/Spiking3.0/Analysis/SpikeAnalysisToolbox/")

import spikeAnalysisToolsV2.data_loading as data
import spikeAnalysisToolsV2.firing_rates as firing
import spikeAnalysisToolsV2.helper as helper
import spikeAnalysisToolsV2.overviews as overview
import spikeAnalysisToolsV2.information_scores as info
import spikeAnalysisToolsV2.plotting as spikeplot
import spikeAnalysisToolsV2.combine_stimuli as combine


#set the masterpath to where the output is saved
masterpath = "/Users/hakunahahannah/Documents/Projects/Spiking3.0/Build/output"

#subfolders = ["20Ex20Id_dakota"+stringC+"tauC"+stringD+"tauD_230_epochs"]
subfolders = ["obj150face450_dakota_230_epochs"]
## if more than the inital epoch is needed *1 needs to be run
extensions = ["initial"] + data.get_epochs(masterpath+"/"+subfolders[0])


In [3]:
#too many epochs makes this computation too large; pick every 10th epoch or so
extensions = extensions[::23]
print(extensions)

object_list = data.load_testing_stimuli_info(masterpath+"/" + subfolders[0]) 
print(object_list)
n_stimuli = np.sum(obj['count'] for obj in object_list)
object_indices = [obj['indices'] for obj in object_list]

stimuli_names = data.load_testing_stimuli_names(masterpath+"/" + subfolders[0])
stimulus_ids = dict()
for i, obj in enumerate(stimuli_names):
    cur_list = stimulus_ids.get(obj, None)
    if cur_list is None:
        stimulus_ids[obj] = [i,]
    else:
        stimulus_ids[obj].append(i)

['initial']
[{'count': 150, 'elements': {'download-162', 'download-60', 'download-1', 'download-70', 'download-138', 'download-53', 'download-166', 'download-47', 'download-43', 'download-150', 'download-18', 'download-27', 'download-41', 'download-173', 'download-49', 'download-62', 'download-121', 'download-171', 'download-35', 'download-40', 'download-58', 'download-28', 'download-125', 'download-167', 'download-88', 'download-81', 'download-123', 'download-159', 'download-142', 'download-126', 'download-163', 'download-89', 'download-112', 'download-119', 'download-78', 'download-141', 'download-87', 'download-16', 'download-33', 'download-45', 'download-50', 'download-98', 'download-2', 'download-80', 'download-23', 'download-170', 'download-20', 'download-155', 'download-65', 'download-48', 'download-9', 'download-113', 'download-74', 'download-129', 'download-79', 'download-92', 'download-102', 'download-145', 'download-34', 'download-25', 'download-146', 'download-172', 'downlo

In [4]:
# info_neurons is just an array of the information from above. This makes it easier to run the functions and pass the information. 
# info_times same for times
network_architecture = dict(
    num_exc_neurons_per_layer = 64*64,
    num_inh_neurons_per_layer = 32*32,
    num_layers = 4
)

info_times = dict(
    length_of_stimulus = 2.0,
    num_stimuli = n_stimuli,
    time_start = 1.5,
    time_end = 1.9
)

In [5]:
spikes = data.load_spikes_from_subfolders(masterpath, subfolders, extensions, False)
print("loading done")
start = timer()
rates_subfolders = firing.calculate_rates_subfolder(
    spikes,
    network_architecture, 
    info_times)
print("Multiprocessing version took {}".format(timer() - start))

spikes = None # we don't need them any more 

Start
loading done


ValueError: cannot convert float NaN to integer

In [33]:
# normalize rates
#firing.inplace_normailize_mean_per_stimulus(rates_subfolders[0])
#raise RuntimeWarning(" !!!!!!!! ALARM FIRING RATES NORMALIZED !!!!!!!!")

#object_indices = list(stimulus_ids.values())
#print(object_indices)
#raise RuntimeWarning("Computing Information for each stimulus seperately now")


[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32], [33], [34], [35], [36], [37], [38], [39]]


In [5]:

folder_of_interest = 0
exc_information, inh_information = info.single_cell_information_all_epochs(rates_subfolders[folder_of_interest], object_indices, 3)

#rates_subfolders = None # save memory
#print(exc_information.shape)
exc_mutual_info  = np.mean(exc_information, axis=1)
#print(exc_mutual_info)
#print(exc_mutual_info.shape)
#print(len(exc_mutual_info.shape))
info=np.expand_dims(exc_mutual_info, axis=1)

Choosen Strategy: firing_rates_to_single_cell_information, || 
   Single Cell information
   :param firing_rates: nested list of shape [stimulus][layer][exc/inh] -> pandas dataframe with fields "ids", "firing_rate"
   :param objects:
   :param n_bins: how many bins the firing rates are sorted into (to make the firing rates discrete)
   :param calc_inhibitory: Flag (to save time)
   :return:
   


In [8]:
item_label=None
mean_of_top_n = 500
threshold = 1

if len(info.shape) != 4:
    info = np.expand_dims(info, axis=1)
    # The case that we have an information score that does not have one value for each item but a combined value
    item_label = ["combined value"]

print(info)

[[[[ 0.          0.          0.         ...,  0.          0.          0.        ]
   [ 0.01007286  0.00669894  0.01687045 ...,  0.00334138  0.          0.00334138]
   [ 0.02373549  0.01687045  0.02373549 ...,  0.00334138  0.01007286
     0.00669894]
   [ 0.00669894  0.02719374  0.01007286 ...,  0.02029445  0.01687045
     0.01346331]]]


 [[[ 0.          0.          0.         ...,  0.          0.          0.        ]
   [ 0.00669894  0.00334138  0.01687045 ...,  0.00334138  0.          0.00334138]
   [ 0.01346331  0.02719374  0.02029445 ...,  0.01346331  0.01007286
     0.01007286]
   [ 0.01346331  0.04120259  0.01007286 ...,  0.02029445  0.01346331
     0.02029445]]]


 [[[ 0.          0.          0.         ...,  0.          0.          0.        ]
   [ 0.01007286  0.00334138  0.01346331 ...,  0.00334138  0.          0.00669894]
   [ 0.02373549  0.01740447  0.02373549 ...,  0.01346331  0.01007286
     0.01346331]
   [ 0.00669894  0.04831516  0.01687045 ...,  0.0306694   0.01199776
 

In [9]:
n_epochs, n_objects, n_layer, n_neurons = info.shape
epochs=np.arange(n_epochs)

if type(epochs) != list and type(epochs) != np.ndarray:
    epochs = list(epochs)

if not item_label:
    item_label = list(["Item {}".format(i) for i in range(n_objects)])
else:
    assert(len(item_label) == n_objects)

if mean_of_top_n == 'all':
    avg_info = np.mean(info, axis=3)
else:
    info_top_n = np.sort(info, axis=3)[:, :, :, -1-mean_of_top_n:]
    avg_info = np.mean(info_top_n, axis=3)
    
print(avg_info)

[[[ 0.06600448  0.03767788  0.02845493  0.02975377]]

 [[ 0.06541617  0.03857986  0.02936552  0.03232679]]

 [[ 0.07081737  0.05310669  0.03389355  0.03473484]]

 [[ 0.07266543  0.14424529  0.1249869   0.10301184]]

 [[ 0.07833775  0.14551608  0.11371186  0.09631499]]]


In [14]:
#so we want to get the highest possible average information in the final layer
fourth_layer = avg_info[:, :, 3]
avg_fourth_layer = (np.mean(fourth_layer))
print('info in 4th layer = ', fourth_layer)

print('avg info across 4th layer = ', avg_fourth_layer)

info in 4th layer =  [[ 0.02975377]
 [ 0.03232679]
 [ 0.03473484]
 [ 0.10301184]
 [ 0.09631499]]
avg info across 4th layer =  0.0592284471475
